In [1]:
from pycelonis import get_celonis
import pandas as pd

In [2]:
url = 'https://bbraun.eu-3.celonis.cloud/'
api_token = 'Y2Q5NWY1ZTctZDk2YS00YWNlLWIwNWEtOGVmYTdiYzg3YTA4OnZKcGxjaEluTUtkSmIvbXZZdk40YnczdFU3UUJKTXJ5MllpQ2xLdGhCRHFJ'

In [3]:
celonis = get_celonis()


2021-02-04 16:48:58 - pycelonis: Login successful! The Application Key currently has access to 688 Analyses and to 4 Data Pools.


In [4]:
celonis.pools


[<Pool, id d1b99aa8-5ded-4410-b2bc-9659f9d53ba6, name SAP ERP>,
<Pool, id e596e1ae-1cec-4dd5-90ea-98b8ae6b99d7, name SAP ERP DEC>,
<Pool, id 2a0c6eba-87d9-4205-9d86-b726f3292659, name MS SQL Celonis>,
<Pool, id 0778113c-cd5f-4502-9ba3-ba7c65649a36, name OpenText - Vendor Invoice Management>,]

In [10]:
pool = celonis.pools.find('d1b99aa8-5ded-4410-b2bc-9659f9d53ba6')

In [11]:
pool.data_connections

[<DataConnection, id 507a7454-3f8b-4598-a904-36886f516085, name USB>,
<DataConnection, id 43200d69-cf72-4943-a9a0-e849c7c3b1bd, name MYB>,
<DataConnection, id 8a3688b3-78f2-4ccf-a00f-6ad01ecfd99c, name BRB>,
<DataConnection, id 26da4b43-22f6-428f-b1e1-f25234904108, name USB-Test>,
<DataConnection, id d52c7f88-d09d-406e-930a-bf94a09cdb78, name DEB>,
<DataConnection, id a52ba2b3-224d-4fb9-bcb1-e25dff382bde, name DEB - test>,
<DataConnection, id b38be222-b287-48e9-a743-f799ed8ae560, name DEA>,
<DataConnection, id 272a6a8b-f885-427f-a20f-4f2bcbc0aa62, name USA>,
<DataConnection, id 5299a972-b4e0-42f8-bb3d-d293210291d8, name MYA>,
<DataConnection, id 366e11f3-c369-4b0e-bc4d-2cba64ee21ec, name BRA>,]

In [12]:
df = pd.DataFrame(pool.tables)

In [13]:
list(set(
df.query('name.str.contains("CBS_")', engine='python').name))


['CBS_P2P_EKKO',
 'CBS_P2P_NAME_MAPPING_COLUMNS',
 'CBS_TMP_P2P_EKPO_EBAN',
 'CBS_P2P_LFA1',
 'CBS_P2P_BSEG',
 'CBS_P2P_LFM1',
 'CBS_TMP_TO',
 'CBS_P2P_RBKP',
 'CBS_TMP_P2P_BKPF_BSEG',
 'CBS_TMP_P2P_EKKO_EKPO',
 'CBS_P2P_MARC',
 'CBS_TMP_P2P_CASES_EKKO',
 'CBS_P2P_EINE',
 'CBS__CEL_P2P_CASES',
 'CBS_P2P_BUKRS_REGION_MAPPING',
 'CBS_P2P_TO',
 'CBS_P2P_MARA',
 'CBS__CEL_P2P_ACTIVITIES',
 'CBS_P2P_EKES',
 'CBS_P2P_EKPO',
 'CBS__CEL_P2P_LINKED_BSEG',
 'CBS__CEL_P2P_ACTIVITY_MASTER_DATA',
 'CBS_P2P_QALS',
 'CBS_P2P_RSEG',
 'CBS_P2P_EBAN',
 'CBS__CEL_P2P_CASES_EKKO',
 'CBS_P2P_T001',
 'CBS_P2P_QPCD',
 'CBS_P2P_QAVE',
 'CBS_P2P_BKPF',
 'CBS_P2P_MSEG',
 'CBS_P2P_NAME_MAPPING_TABLES',
 'CBS_P2P_QPGR',
 'CBS_P2P_LFB1',
 'CBS_P2P_EKET',
 'CBS_P2P_CONTRACT_INFO',
 'CBS_P2P_QPAM',
 'CBS_P2P_EKBE']

In [33]:
import sys
from os.path import dirname, abspath
#_PARENT_DIR = dirname(dirname(dirname(abspath(__file__))))
#sys.path.append(_PARENT_DIR)
#from utils._utils import parse_celonis_url, get_user_data, get_logger              ### <--
from pycelonis import get_celonis
import argparse
#logger = get_logger()
def unify_tables(url, api_token, datapool, tables, tableSchema, function = 'null'):
    
    #loop through defined tables
    #for table_name in tables:
       
    c = get_celonis(url, api_token)
    #get_user_data(c, 'Table Unifier')
    #url_options= parse_celonis_url(url)                                            ### <--
    #pool = c.pools.find(url_options['id'])                                         ### <--
    
    pool = c.pools.find(datapool)                                                   ### <--

    minReq = []
    maxReq = []
    sameTables = []
    schemalist = tableSchema                                                        # added to extract required table schemas (in python list format)
    columns = ''
    select = ''
    
    # create or find global data job 
    try:
        global_dj = pool.data_jobs.find('TEST123')
    except:
        global_dj = pool.create_data_job(name = 'TEST123')
    
    #loop through defined tables
    for table_name in tables:
    
        # find tables to unify
        x = 0
        for schema in global_dj.tables:
            for table in schema:
                x = x + 1
                if table['schemaName'] not in schemalist:
                    continue
                if table['name'] == table_name:
                    sameTables.append(table)
        
       
        
        
        if len(sameTables) == 0:
            print('TABLE', table_name, 'CAN NOT BE UNIFIED! Not existing in source systems!')
         
            try: 
                global_dj.transformations.find("Unify " + table_name).delete()
                global_dj.create_transformation(name = "Unify " + table_name, statement = '-- no existing tables to unify in defined source systems')
                continue
            except:
                global_dj.create_transformation(name = "Unify " + table_name, statement = '-- no existing tables to unify in defined source systems')
                continue
        else:
            # create minimal column requirement
            for col in sameTables[0]['columns']:
                try:
                    if col in sameTables[1]['columns']:                                              # ERROR CHECKING !
                        minReq.append(col)
                except:
                    continue

            """
            #only required if function = 'min'

            if len(sameTables) > 2:
                for table in sameTables[2:]:
                    for col in minReq:
                        if col not in table['columns']:
                            minReq.pop(col)
                # create transformation statement for minimal table requirement

            if function == 'min':
                for col in minReq:
                    if(minReq.index(col) == 0):
                        columns += '{}\n'.format(col['name'])
                    else:
                        columns += ',{}\n'.format(col['name'])
                for table in sameTables:
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            """            

            # create transformation for statement for extension with null
            if function == 'null':
                #columns = ''
                #select = ''
                
                for table in sameTables:
                    for col in table['columns']:
                        if col not in maxReq:
                            maxReq.append(col)
                for table in sameTables:
                    for col in maxReq:
                        if(maxReq.index(col) == 0):
                            columns += "'" + table['schemaName'] + "' " + 'AS "SCHEMA"\n' + ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                        else:
                            columns += ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    columns = ''
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            # create transformation
            #statement = 'DROP VIEW IF EXISTS "{}_UNIFIED";\n\nCreate VIEW "{}_UNIFIED" AS(\n{}\n);'.format(table_name, table_name, select)
            statement = 'DROP VIEW IF EXISTS "{}";\n\nCreate VIEW "{}" AS(\n{}\n);'.format(table_name, table_name, select)                      ### <-- Unified
            try: 
                    global_dj.transformations.find("Unify " + table_name).delete()
            except: 
                pass
            
            global_dj.create_transformation(name = "Unify " + table_name, statement = statement)
            print('The union statement has been saved in the global data job "Unify Tables" for Table', table_name)
            select = ''
            sameTables = []
            minReq = []
            maxReq = []
            
            # reminder: datatypes

In [36]:

# URL to the Cloud Data Pool where the tables to unify are hosted.
url = 'https://bbraun.eu-3.celonis.cloud/integration/ui/pools/d1b99aa8-5ded-4410-b2bc-9659f9d53ba6'

# Specify a valid API token for your Cloud Team.
api_token = 'MmNjYzZiMzctZDhhOS00ZDRlLTkxZWMtYzg2NDM5NjUzN2E4OkgrSExiaWtiQWsvbWRkOU0zTEltVXlPMmdGUkZLajROa2diYytBVlZXa3lu'

# Specify a datapool 
datapool = 'd1b99aa8-5ded-4410-b2bc-9659f9d53ba6'

# Specify the table names for which the tables should be merged. 
#tables = ['VBEH']

tables = ['CBSO_O2C_KNA1'
 ]

# Specify 'min' if the unified table should be the minimal column requirement and 'null' if the not existing columns should be filled with null values.
function = 'null'

# Specify the required table schemas in a python list
tableSchema = ['d1b99aa8-5ded-4410-b2bc-9659f9d53ba6_272a6a8b-f885-427f-a20f-4f2bcbc0aa62',
               'd1b99aa8-5ded-4410-b2bc-9659f9d53ba6_366e11f3-c369-4b0e-bc4d-2cba64ee21ec',
               'd1b99aa8-5ded-4410-b2bc-9659f9d53ba6_5299a972-b4e0-42f8-bb3d-d293210291d8',
               'd1b99aa8-5ded-4410-b2bc-9659f9d53ba6_b38be222-b287-48e9-a743-f799ed8ae560']
unify_tables(url = url, api_token = api_token, datapool = datapool, tables = tables, tableSchema = tableSchema, function = function)

2021-02-04 15:03:08 - pycelonis: Login successful! Hello Application Key, this key currently has access to 688 analyses.
2021-02-04 15:03:18 - pycelonis: Best matches: [(0.98, 'Unify CBSO_O2C_KNB1'), (0.96, 'Unify CBSO_O2C_NAST'), (0.96, 'Unify CBSO_O2C_KNKK')]
The union statement has been saved in the global data job "Unify Tables" for Table CBSO_O2C_KNA1
